In [15]:
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.colors as mcolors
from scipy.stats import bootstrap
import random as rd
from file_helper import load_all_files
# sketchy way of figuring out if latex is installed, might fail.
# If it does, comment out
from distutils.spawn import find_executable
from itertools import cycle
import os
import pickle as pk


In [16]:
path_dir = 'results'
list_files = load_all_files(path_dir)
print(list_files)
for file in list_files:
    file_name = file.split('.')[0]
    if 'binning' in file_name:
        print(file_name)
        with open(os.path.join(path_dir, file), 'rb') as f:  # load the data
                data = pk.load(f)

['GEN_46656_10000_2_2_3_S_algo.pk', 'GEN_46656_10000_10_2_3_ranking.pk', 'SYNTH_46656_1000_50_3_2_binning_algo.pk', 'SYNTH_46656_1000_50_3_2_ranking.pk', 'SYNTH_46656_1000_50_3_2_hypothesis_algo.pk', 'GEN_46656_10000_10_2_3_hypothesis_algo.pk', 'GEN_10000000000_10000_2_2_3_S_algo.pk', 'GEN_46656_10000_2_2_3_hypothesis_algo.pk', 'GEN_46656_10000_10_2_3_S_algo.pk', 'GEN_46656_10000_2_2_3_ranking.pk', 'GEN_10000000000_10000_2_2_3_ranking.pk', 'GEN_10000000000_10000_2_2_3_hypothesis_algo.pk', 'GEN_10000000000_10000_2_2_3_binning_algo.pk', 'SYNTH_46656_1000_50_3_2_S_algo.pk']
SYNTH_46656_1000_50_3_2_binning_algo
GEN_10000000000_10000_2_2_3_binning_algo


In [17]:
baseline_name = list(data['data'].keys())
baseline_name

['ground truth']

In [20]:
data['data'][baseline_name[0]]

[[{'p': [0.7500000000153644, 0.2500000000107812, 0],
   'q': [0.7523999999999335, 0.24759999999998905, 7.749356711883593e-14]},
  {'p': [0.7500000000153644, 0.2500000000107812, 0],
   'q': [0.747999999999934, 0.25199999999998857, 7.738254481637341e-14]}],
 [{'p': [0.7468944279251288, 0.003105572089946579, 0.24965911597194884, 0],
   'q': [0, 0.7523999999999335, 0, 7.749356711883593e-14]},
  {'p': [0.7469146825547603, 0.003085317460316959, 0.249653053361719, 0],
   'q': [0, 0.747999999999934, 0, 7.738254481637341e-14]}],
 [{'p': [0.7468944279251288,
    0.003105572089946579,
    0.24965911597194884,
    0.0003408840388007138,
    0],
   'q': [0,
    0.7523999999999335,
    0,
    0.24759999999998905,
    7.749356711883593e-14]},
  {'p': [0.7469146825547603,
    0.003085317460316959,
    0.249653053361719,
    0.0003469466490299909,
    0],
   'q': [0,
    0.747999999999934,
    0,
    0.25199999999998857,
    7.738254481637341e-14]}],
 [{'p': [0.7468944279251288,
    0.00310557208994657